# Chapter 6 - Hybrid Recommendation using LightFM

<div style="text-align:center;">
    <img src='images/hr.png' width='800'>
</div>

Collaborative filtering is the predictive process behind recommendation engines. Recommendation engines analyze information about users with similar tastes to assess the probability that a target individual will enjoy something.

Collaborative filtering uses algorithms to filter data from user reviews to make personalized recommendations for users with similar preferences. Collaborative filtering is also used to select content and advertising for individuals on social media.

Collaborative filtering filters information by using the interactions and data collected by the system from other users. For example when we want to find a new movie to watch we'll often ask our friends for recommendations.

Naturally, we have greater trust in the recommendations from friends who share tastes similar to our own. Collaborative filtering does the same job. Collaborative filtering m**ostly focuses on finding similarity between** users and recommend each other their likes. There are various ways to find the similarity meas*ure : Cosine simi*la*rity, Pearson simi*la*rity, Jaccard simi*larity etc.

In [1]:
# Importing basic libraries
import pandas as pd
import numpy as np

# for constructing sparse matrix
from scipy.sparse import coo_matrix 

# lightfm 
from lightfm import LightFM
from lightfm.evaluation import auc_score

#sklearn
from sklearn.model_selection import train_test_split

# timing
import time

import warnings
warnings.filterwarnings("ignore")

C:\Users\lsoares\AppData\Local\anaconda3\Lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


### Creating the Dataset

In [2]:
#read csv data

order_df = pd.read_excel('data/Rec_sys_data.xlsx','order')
customer_df = pd.read_excel('data/Rec_sys_data.xlsx','customer')
product_df = pd.read_excel('data/Rec_sys_data.xlsx','product')

In [3]:
order_df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,DeliveryDate,Discount%,ShipMode,ShippingCost,CustomerID
0,536365,84029E,6,2010-12-01 08:26:00,2010-12-02 08:26:00,0.20,ExpressAir,30.12,17850
1,536365,71053,6,2010-12-01 08:26:00,2010-12-02 08:26:00,0.21,ExpressAir,30.12,17850
2,536365,21730,6,2010-12-01 08:26:00,2010-12-03 08:26:00,0.56,Regular Air,15.22,17850
3,536365,84406B,8,2010-12-01 08:26:00,2010-12-03 08:26:00,0.30,Regular Air,15.22,17850
4,536365,22752,2,2010-12-01 08:26:00,2010-12-04 08:26:00,0.57,Delivery Truck,5.81,17850


In [4]:
customer_df.head()

,CustomerID,Gender,Age,Income,Zipcode,Customer Segment
0,13089,male,53,High,8625,Small Business
1,15810,female,22,Low,87797,Small Business
2,15556,female,29,High,29257,Corporate
3,13137,male,29,Medium,97818,Middle class
4,16241,male,36,Low,79200,Small Business


In [5]:
product_df.head()

,StockCode,Product Name,Description,Category,Brand,Unit Price
0,22629,Ganma Superheroes Ordinary Life Case For Samsu...,"New unique design, great gift.High quality pla...",Cell Phones|Cellphone Accessories|Cases & Prot...,Ganma,13.99
1,21238,Eye Buy Express Prescription Glasses Mens Wome...,Rounded rectangular cat-eye reading glasses. T...,Health|Home Health Care|Daily Living Aids,Eye Buy Express,19.22
2,22181,MightySkins Skin Decal Wrap Compatible with Ni...,Each Nintendo 2DS kit is printed with super-hi...,Video Games|Video Game Accessories|Accessories...,Mightyskins,14.99
3,84879,Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lac...,The sheerest compression stocking in its class...,Health|Medicine Cabinet|Braces & Supports,Medi,62.38
4,84836,Stupell Industries Chevron Initial Wall D cor,Features: -Made in the USA. -Sawtooth hanger o...,Home Improvement|Paint|Wall Decals|All Wall De...,Stupell Industries,35.99


In [6]:
#Merging the datasets

merged_df = pd.merge(order_df,customer_df,left_on=['CustomerID'], right_on=['CustomerID'], how='left')
merged_df = pd.merge(merged_df,product_df,left_on=['StockCode'], right_on=['StockCode'], how='left')

merged_df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,DeliveryDate,Discount%,ShipMode,ShippingCost,CustomerID,Gender,Age,Income,Zipcode,Customer Segment,Product Name,Description,Category,Brand,Unit Price
0,536365,84029E,6,2010-12-01 08:26:00,2010-12-02 08:26:00,0.20,ExpressAir,30.12,17850,female,48,Medium,84306,Middle class,"3 1/2""W x 20""D x 20""H Funston Craftsman Smooth...",Our Rustic Collection is an instant classic. O...,Home Improvement|Hardware|Brackets and Angle I...,Ekena Milwork,199.11
1,536365,71053,6,2010-12-01 08:26:00,2010-12-02 08:26:00,0.21,ExpressAir,30.12,17850,female,48,Medium,84306,Middle class,Awkward Styles Shamrock Flag St. Patrick's Day...,Our St Patrick's Day Collection is perfect for...,Clothing|Men|Mens T-Shirts & Tank Tops|Mens Gr...,Awkward Styles,23.95
2,536365,21730,6,2010-12-01 08:26:00,2010-12-03 08:26:00,0.56,Regular Air,15.22,17850,female,48,Medium,84306,Middle class,Ebe Men Black Rectangle Half Rim Spring Hinge ...,Count on EBE for all of your eye correction ne...,Health|Home Health Care|Daily Living Aids,Eye Buy Express,26.99
3,536365,84406B,8,2010-12-01 08:26:00,2010-12-03 08:26:00,0.30,Regular Air,15.22,17850,female,48,Medium,84306,Middle class,MightySkins Skin Decal Wrap Compatible with Ap...,Mightyskins are removable vinyl skins for prot...,Electronics|Electronics Learning Center|Ads Fr...,Mightyskins,14.99
4,536365,22752,2,2010-12-01 08:26:00,2010-12-04 08:26:00,0.57,Delivery Truck,5.81,17850,female,48,Medium,84306,Middle class,awesome since 1948 - 69th birthday gift t-shir...,awesome since 1948 - 69th birthday gift t-shir...,Clothing|Men|Mens T-Shirts & Tank Tops|Mens T-...,Shirtinvaders,49.33


In [7]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272404 entries, 0 to 272403
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   InvoiceNo         272404 non-null  int64         
 1   StockCode         272404 non-null  object        
 2   Quantity          272404 non-null  int64         
 3   InvoiceDate       272404 non-null  datetime64[ns]
 4   DeliveryDate      272404 non-null  datetime64[ns]
 5   Discount%         272404 non-null  float64       
 6   ShipMode          272404 non-null  object        
 7   ShippingCost      272404 non-null  float64       
 8   CustomerID        272404 non-null  int64         
 9   Gender            272404 non-null  object        
 10  Age               272404 non-null  int64         
 11  Income            272404 non-null  object        
 12  Zipcode           272404 non-null  int64         
 13  Customer Segment  272404 non-null  object        
 14  Prod

### Unique Users

In [8]:
# Creating the list of unique users
def unique_users(data, column):   
    return np.sort(data[column].unique())

# Creating the list of unique produts
def unique_items(data, column):
    item_list = data[column].unique()
    return item_list

def features_to_add(customer, column1,column2,column3):
    customer1 = customer[column1]
    customer2 = customer[column2]
    customer3 = customer[column3]
    return pd.concat([customer1,customer3,customer2], ignore_index = True).unique()

# Create id mappings to convert user_id, item_id, and feature_id
def mapping(user_list, item_list, feature_unique_list):
    #creating empty output dicts
    user_to_index_mapping = {}
    index_to_user_mapping = {}
    # Create id mappings to convert user_id
    for user_index, user_id in enumerate(user_list):
        user_to_index_mapping[user_id] = user_index
        index_to_user_mapping[user_index] = user_id
        
    item_to_index_mapping = {}
    index_to_item_mapping = {}
    # Create id mappings to convert item_id
    for item_index, item_id in enumerate(item_list):
        item_to_index_mapping[item_id] = item_index
        index_to_item_mapping[item_index] = item_id
        
    feature_to_index_mapping = {}
    index_to_feature_mapping = {}
    # Create id mappings to convert feature_id
    for feature_index, feature_id in enumerate(feature_unique_list):
        feature_to_index_mapping[feature_id] = feature_index
        index_to_feature_mapping[feature_index] = feature_id
        
    return user_to_index_mapping, index_to_user_mapping, \
           item_to_index_mapping, index_to_item_mapping, \
           feature_to_index_mapping, index_to_feature_mapping

In [9]:
# create the user, item, feature lists
user_list = unique_users(order_df, "CustomerID")
item_list = unique_items(product_df, "Product Name")
feature_unique_list = features_to_add(customer_df,'Customer Segment',"Age","Gender")

user_list

array([12346, 12347, 12348, ..., 18282, 18283, 18287], dtype=int64)

In [10]:
#item_list

In [11]:
feature_unique_list

array(['Small Business', 'Corporate', 'Middle class', 'male', 'female',
       53, 22, 29, 36, 48, 45, 47, 23, 39, 34, 52, 51, 35, 19, 26, 37, 18,
       20, 21, 41, 31, 28, 50, 38, 30, 25, 32, 55, 43, 54, 49, 40, 33, 44,
       46, 42, 27, 24], dtype=object)

In [12]:
# generate mapping, LightFM library can't read other than (integer) index
user_to_index_mapping, index_to_user_mapping, \
           item_to_index_mapping, index_to_item_mapping, \
           feature_to_index_mapping, index_to_feature_mapping = mapping(user_list, item_list, feature_unique_list)

In [13]:
#user_to_index_mapping

In [14]:
user_to_product = merged_df[['CustomerID','Product Name','Quantity']]
user_to_product = user_to_product.groupby(['CustomerID','Product Name']).agg({'Quantity':'sum'}).reset_index()

user_to_product.tail()

,CustomerID,Product Name,Quantity
138397,18287,Sport-Tek Ladies PosiCharge Competitor Tee,24
138398,18287,Ultra Sleek And Spacious Pearl White Lacquer 1...,6
138399,18287,"Union 3"" Female Ports Stainless Steel Pipe Fit...",12
138400,18287,awesome since 1948 - 69th birthday gift t-shir...,4
138401,18287,billyboards Porcelain Menu Chalkboard,6


In [15]:
product_to_feature = merged_df[['Product Name','Customer Segment','Quantity']]
product_to_feature = product_to_feature.groupby(['Product Name','Customer Segment']).agg({'Quantity':'sum'}).reset_index()

product_to_feature.head()

,Product Name,Customer Segment,Quantity
0,"""In Vinyl W.e Trust"" Rasta Quote Men's T-shirt",Corporate,712
1,"""In Vinyl W.e Trust"" Rasta Quote Men's T-shirt",Middle class,272
2,"""In Vinyl W.e Trust"" Rasta Quote Men's T-shirt",Small Business,388
3,"""Soccer"" Vinyl Graphic - Large - Ivory",Corporate,1940
4,"""Soccer"" Vinyl Graphic - Large - Ivory",Middle class,1418


### Train Test Split

In [16]:
user_to_product_train,user_to_product_test = train_test_split(user_to_product,test_size=0.33, random_state=42)

In [17]:
print("Training set size:")
print(user_to_product_train.shape)

print("\nTest set size:")
print(user_to_product_test.shape)

Training set size:
(92729, 3)

Test set size:
(45673, 3)


In [18]:
def interactions(data, row, col, value, row_map, col_map):
    
    #converting the row with its given mappings
    row = data[row].apply(lambda x: row_map[x]).values
    #converting the col with its given mappings
    col = data[col].apply(lambda x: col_map[x]).values
    value = data[value].values
    #returning the interaction matrix
    return coo_matrix((value, (row, col)), shape = (len(row_map), len(col_map)))

In [19]:
# generate user_item_interaction_matrix for train data
user_to_product_interaction_train = interactions(user_to_product_train, "CustomerID", 
                                                    "Product Name", "Quantity", user_to_index_mapping, item_to_index_mapping)

# generate item_to_feature interaction
product_to_feature_interaction = interactions(product_to_feature, "Product Name", "Customer Segment","Quantity",item_to_index_mapping, 
                                              feature_to_index_mapping)

# generate user_item_interaction_matrix for test data
user_to_product_interaction_test = interactions(user_to_product_test, "CustomerID","Product Name", "Quantity", user_to_index_mapping,
                                                item_to_index_mapping)

In [20]:
#print(user_to_product_interaction_train)

### Model building on training set
Parameters: Loss=warp epochs=1 num_threads=4

In [ ]:
# initialising model with warp loss function
model_with_features = LightFM(loss = "warp")

# fitting the model with hybrid collaborative filtering + content based (product + features)
start = time.time()
#===================


model_with_features.fit_partial(user_to_product_interaction_train,
          user_features=None, 
          item_features=product_to_feature_interaction, 
          sample_weight=None, 
          epochs=1, 
          num_threads=4,
          verbose=False)

#===================
end = time.time()
print("time taken = {0:.{1}f} seconds".format(end - start, 2))

In [ ]:
start = time.time()
#===================
auc_with_features = auc_score(model = model_with_features, 
                        test_interactions = user_to_product_interaction_test,
                        train_interactions = user_to_product_interaction_train, 
                        item_features = product_to_feature_interaction,
                        num_threads = 4, check_intersections=False)
#===================
end = time.time()
print("time taken = {0:.{1}f} seconds".format(end - start, 2))

print("average AUC without adding item-feature interaction = {0:.{1}f}".format(auc_with_features.mean(), 2))

### Model building on training set
Parameters: Loss=logistic epochs=10 num_threads=20

In [ ]:
model_with_features = LightFM(loss = "logistic")

# fitting the model with hybrid collaborative filtering + content based (product + features)
start = time.time()
#===================


model_with_features.fit_partial(user_to_product_interaction_train,
          user_features=None, 
          item_features=product_to_feature_interaction, 
          sample_weight=None, 
          epochs=10, 
          num_threads=20,
          verbose=False)

#===================
end = time.time()
print("time taken = {0:.{1}f} seconds".format(end - start, 2))

In [ ]:
start = time.time()
#===================
auc_with_features = auc_score(model = model_with_features, 
                        test_interactions = user_to_product_interaction_test,
                        train_interactions = user_to_product_interaction_train, 
                        item_features = product_to_feature_interaction,
                        num_threads = 4, check_intersections=False)
#===================
end = time.time()
print("time taken = {0:.{1}f} seconds".format(end - start, 2))

print("average AUC without adding item-feature interaction = {0:.{1}f}".format(auc_with_features.mean(), 2))

In [ ]:
def train_test_merge(training_data, testing_data):

    # initialising train dict
    train_dict = {}
    for row, col, data in zip(training_data.row, training_data.col, training_data.data):
        train_dict[(row, col)] = data

    # replacing with the test set

    for row, col, data in zip(testing_data.row, testing_data.col, testing_data.data):
        train_dict[(row, col)] = max(data, train_dict.get((row, col), 0))


    # converting to the row
    row_list = []
    col_list = []
    data_list = []
    for row, col in train_dict:
        row_list.append(row)
        col_list.append(col)
        data_list.append(train_dict[(row, col)])

    # converting to np array

    row_list = np.array(row_list)
    col_list = np.array(col_list)
    data_list = np.array(data_list)

    return coo_matrix((data_list, (row_list, col_list)), shape = (training_data.shape[0], training_data.shape[1]))

In [ ]:
user_to_product_interaction = train_test_merge(user_to_product_interaction_train, user_to_product_interaction_test)

user_to_product_interaction

### Final Model after combining the train and test data
Parameters: Loss=warp epochs=10 num_threads=20

In [ ]:
# retraining the final model with combined dataset

final_model = LightFM(loss = "warp",no_components=30)

# fitting to combined dataset

start = time.time()
#===================

final_model.fit(user_to_product_interaction,
          user_features=None, 
          item_features=product_to_feature_interaction, 
          sample_weight=None, 
          epochs=10, 
          num_threads=20,
          verbose=False)

#===================
end = time.time()
print("time taken = {0:.{1}f} seconds".format(end - start, 2))

In [ ]:
def get_recommendations(model,user,items,user_to_product_interaction_matrix,user2index_map,product_to_feature_interaction_matrix):

# getting the userindex

    userindex = user2index_map.get(user, None)

    if userindex == None:
        return None

    users = userindex

    # products already bought

    known_positives = items[user_to_product_interaction_matrix.tocsr()[userindex].indices]
    print('User index =',users)

    # scores from model prediction
    scores = model.predict(user_ids = users, item_ids = np.arange(user_to_product_interaction_matrix.shape[1]),
                           item_features=product_to_feature_interaction_matrix)

    # top items

    top_items = items[np.argsort(-scores)]

    # printing out the result
    print("User %s" % user)
    print("     Known positives:")

    for x in known_positives[:10]:
        print("                  %s" % x)


    print("     Recommended:")

    for x in top_items[:10]:
        print("                  %s" % x)

In [ ]:
get_recommendations(final_model,17017,items,user_to_product_interaction,user_to_index_mapping,
                    product_to_feature_interaction)

In [ ]:
get_recommendations(final_model,18287,items,user_to_product_interaction,user_to_index_mapping,
                    product_to_feature_interaction)

In [ ]:
get_recommendations(final_model,13933,items,user_to_product_interaction,user_to_index_mapping,
                    product_to_feature_interaction)